<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <h2 style="color:#ff7800;">Autonomous Traders</h2>
            <span style="color:#ff7800;">An equity trading simulation to illustrate autonomous agents powered by tools and resources from MCP servers.
            </span>
        </td>
    </tr>
</table>

### Week 6 Day 4

And now - introducing the Capstone project:


# Autonomous Traders

An equity trading simulation, with 4 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Our home-made Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data

And a resource to read information about the trader's account, and their investment strategy.

The goal of today's lab is to make a new python module, `traders.py` that will manage a single trader on our trading floor.

We will experiment and explore in the lab, and then migrate to a python module when we're ready.

---

We are going to create a simulation to illustrate autonomous agents powered by tools and resources from MCP servers. We are going to have four different traders, eventually. And one researcher, actually each trader will have their own researcher, powered by a bunch of MCP servers. We are going to have our homemade accounts MCP server that you remember that we did in the second day. We will use Fetch that we used in the first day, we will use Memory, we will use the SQL-based relationship Memory that we looked at, we will use the Brave Search, and we will use the financial data courtesy of Polygon.io. So what we are going to go through now is build this in the lab, and then we are going to look at the code, the module traders.py that will take the same thing pulled together. And it shows you a practice which I like to do, which is to work initially in the lab while you experiment. And it ties to a point that I know I make a million times that I'll make again at the is how important it is to approach agent projects with a data scientist's hat on, first and foremost. Be looking to experiment and understand what you're doing, not just jumping straight into engineering and building, building. It's important to start by investigating and understanding what you're doing. One other thing that's important to do is not use this for trading decisions. But I already mentioned that. So one more warning for that.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <h2 style="color:#ff7800;">One more time --</h2>
            <span style="color:#ff7800;">Please do not use this for actual trading decisions!!
            </span>
        </td>
    </tr>
</table>

In [13]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

### Let's start by gathering the MCP params for our trader

Hey, let's get going. Let's do our imports and set our .emv, and let's look at the Polygon situation. Do we have an API key, and have you said whether or not we are in a paid plan? We are in a paid plan for me. You may be false there, and we're not using the real-time Polygon.

In [14]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


Okay. What we're going to be doing in the next few cells is collecting together the different MCP server parameters. For all of the MCP servers that we're going to equip our model with. So first of all, I've got a little decision here. If we're using any of the paid Polygon plans, then we are going to directly use Polygon's MCP server with its whole set of different tools. If not, we're just going to use that tiny MCP server that I handcrafted in marketserver.py. And that's because if you're using the free plan, we don't want to overwhelm the model with lots of different tools. And there's another reason for it too, which is that here I'm using that trick of caching the previous day's data so that we don't exceed our rate limits with Polygon on the free plan. So that's why we've got a little decision there. And then we're also going to include in our parameters the accounts server. So this is our homegrown MCP server to read and write from accounts. And then there's a new one, push server. I wonder what that could possibly be. Let's just run this, and then we're going to take a quick look at push server. It's another homegrown MCP server. What could it possibly be doing? Let's have a look. 

In [15]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

---

### Here it is, `push_server.py`

```py
import os
from dotenv import load_dotenv
import requests
from pydantic import BaseModel, Field
from mcp.server.fastmcp import FastMCP

load_dotenv(override=True)

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"


mcp = FastMCP("push_server")


class PushModelArgs(BaseModel):
    message: str = Field(description="A brief message to push")


@mcp.tool()
def push(args: PushModelArgs):
    """Send a push notification with this brief message"""
    print(f"Push: {args.message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": args.message}
    requests.post(pushover_url, data=payload)
    return "Push notification sent"


if __name__ == "__main__":
    mcp.run(transport="stdio")
```

Push server, of course, it's got a push notification. You know I like the push notifications because it makes it feel so autonomous when it's messaging you suddenly. And so we're going to have it pushing. We're going to arm it with an MCP tool. This is an example of a case where we're writing this MCP server and we're not delegating on to some business logic. We're just putting all the logic right here in this single Python module. It's an MCP server, it has a single tool, it's called push, and it takes a little Pydantic object that we've set up here to say that it is a brief message to push, and that is the message that we give. So this is a clear example of how you can write a little tool and expose it as an MCP server. Now, as I said before, really there's absolutely nothing stopping you from just having this as a tool. In fact, that would be the better way of doing it. There's no point in having an MCP server like this when we're just using the tool ourselves locally. But we want to get in the practice of making MCP servers, so why not?

### And now for our researcher

Okay, so with that, we have now also included our push notification as well as the accounts and the market data as our trader MCP servers. So that's set up the MCP servers that our trader will use, but we'll have another agent called the researcher that's able to do market research, and we also want to arm that agent with tools as well. And for that one, we will use the brave key, and we'll give it fetch as well. We'll give it the ability to fetch webpages, and so both of these two together are going to be some of the tools that we will equip our model with. Alright, and now it's going to be time to put these to use.

In [16]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

### Now create the MCPServerStdio for each

Okay, so we've gathered up our parameters into these params lists. What we're now going to do is MCP servers. We're going to instantiate MCP servers with each of those params and with the 30-second timeout. And so we do all of that, and we just built a bunch of these MCP servers ready to go.

In [17]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

Okay, so we're going to have two different agents that we're going to define. We're going to have a trader that's able to make trading decisions and a researcher that does market research. And the trader will use that researcher. And you may remember from Week 2 in OpenAI Agents SDK that when you want to have that kind of collaboration where one agent uses another, the best way to do it is to have that other agent, the research agent, be like a tool. Convert it into a tool so that that agent can just be used as a tool by the trader agent. And that's exactly what we're going to do now. So we start by defining our researcher agent. So here's the system prompt, the instruction. You're a financial researcher. You search the web for interesting news, and then you carry out deeper research and respond with your findings. And we tell it the current date. And you remember I said it's better to do this than to have a tool to look up the date, because you might as well always pass it on and not add extra complexity to the agent to force it to come back and run a tool. So we just provide the current date right in there. And then we pass in, of course, our MCP service, and that defines our researcher.

In [18]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

And then you'll remember this construct. This is how we say we want to use this as a tool. We simply call as tool on that researcher, and we give the tool a name and a description. And that means that this agent will be available for use by other agents that want to be able to treat this like it's a tool. And that is a very common pattern with OpenAI Agents SDK. Well, before we use the research agent as a tool, let's just try calling it directly to check it works.

In [19]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

So we're going to ask the question, what's the latest news on Amazon? And we're going to, first of all, go through and connect with all of our MCP servers. Now, you'll notice this is a bit different. I normally have, like, with MCP Studio as, and then I do it that way with a context manager. So why am I doing it differently this time? Because if you've got a bunch of these, we'd have to have lots of widths all nested, and that would be quite clunky. And so this is just another way of doing it. You should also clean up if you do this, but because we're just running in a JupyterLab anyway, it doesn't really matter. So we're connecting to the server here. We're then getting our research agent, and then we're calling runner.run for the research agent, asking the agent and the question. And this is new. I'm also passing in something called max turns in here. And that's because if I, the default is 10, which means that it can, by default, do up to 10 sets of tool calls. But if we wanted to do deep research, we might want it to take longer than that. So we might want to give it up to 30 possible maximum turns. So that's why I'm setting it that way, and it's good to know that's another thing that you can control. And you could also, of course, have that be a smaller number if you don't want to let your agents go off and potentially get into a loop of overthinking about things. But 30 seems to have done the trick for us. It's probably really nearly like that. And we've got back a bunch of information about Amazon.

In [20]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



Latest news on Amazon includes the announcement of their second quarter 2025 results:

- Net sales increased 13% to $167.7 billion compared to $148.0 billion in Q2 2024. Adjusting for currency exchange impacts, sales rose by 12%.
- Amazon's online stores sales grew 11% year over year to $61.5 billion, exceeding Wall Street expectations of $59 billion.
- Seller services revenue increased 11% year over year to $40.3 billion, also beating analyst projections.
- Amazon expects capital expenditures of about $105 billion in 2025, signaling heavy investments.
- Despite Q2 earnings beat, the stock fell somewhat due to mixed guidance and increased competition, particularly in advertising and AI sectors.

This shows Amazon's strong top-line growth and robust revenue streams, alongside significant investments for future growth, but also some investor caution around competition and guidance clarity. Would you like a deeper dive into any specific aspect?

### Look at the trace

Let's go in and look at the trace to see what happened behind the scenes. So if we come in to the trace and we come in to the researcher, you'll see that it did a brave search. It did a bunch of fetching of web pages. It did another brave search and some more fetching of web pages before responding with its answer. And you should do this and go back and have a look and see what it does and what kind of searching and fetching the agent is doing as part of its research into Amazon.

https://platform.openai.com/traces

---

Okay, so now let's look at our trader. So I'm going to start by giving our trader a strategy because that's something that we store in the account. And the reason that I give a trader a strategy which is stored is that I want traders to be able to change that strategy should they wish. We're going to give each of our traders a unique strategy to set them going, but we want to give them some autonomy to choose to evolve their strategy if they want to. But for me, Ed's initial strategy, I'm going to be a day trader that aggressively buys and sells shares based on news and market conditions. And I'm going to call this reset function to get Ed off to a good start. And let's use these resources to read my account and my strategy at this starting position. So here we go. My starting position is I have $10,000 ready to invest. There is my description and I have empty transactions and nothing in my portfolio and everything is ready for business. It's now time to create our trader agent that is going to take this persona and be able to make trades as a result.

In [21]:
ed_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Ed").reset(ed_initial_strategy)

display(Markdown(await read_accounts_resource("Ed")))
display(Markdown(await read_strategy_resource("Ed")))

{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-08-18 10:55:55", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

Okay, so this is our trader agent right here. It's called Ed and the account details is reading in a resource and the strategy is reading a resource. What is this reading a resource? This is of course calling the MCP client that we created ourselves a couple of days ago. It's calling an MCP client that then calls the MCP server that provides the resource by calling our business logic. So it's kind of cool. We're using this resource side of MCP. The fact that you don't just need to use MCP for tools, you can use it for resources as well. And so what does it mean to use MCP for resources? What do you do with these resources? Well, it's just text that you shove in the prompt. You just add it to the prompt to give your agent more context to be able to make his decisions. And that's exactly what we do here. So we call these two resources and then we put together our system prompt. You're a trader that manages a portfolio of shares. Your name's Ed, your account's under your name, you have access to tools to do your job. Your investment strategy is, and then we shove in the strategy from this service call here. Your account, your current holdings and balance is, and we shove in the account details right here. And then we tell it to make decisions based on its tools. So if we run that and we print the instructions, we'll see that when it prints out, we get things like the current holdings and balance included in our prompt. This is our resource. And you can see I'm just shoving JSON in there because LLMs love JSON. It's going to be great with this. It'll make total sense to it.

In [22]:
agent_name = "Ed"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [23]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is Ed and your account is under your name, Ed.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-08-18 10:55:55", 10000.0], ["2025-08-18 10:55:56", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please

### And to run our Trader

And so with that, we can kick off our trader. So it's running while I speak. So we connect to each of the MCP servers. We then turn our research agent into a tool and we get our research agent as a tool. We then create our trader agent. We give it its instructions. We pass in as tools, we pass in our researcher tool, which is a wrapper around the agent. For MCP servers, we pass in the full set of MCP servers. We're using GPT-40 mini in here. You might want to update that to GPT-41 mini if you're looking at that now. And then we call runner.run and we pass in the trader and the prompt. And again, I'm adding to max terms. I'm not going with a default of 10. I'm giving it up to 30 terms to really go to town and be requesting across things. And now, typically, this takes a minute. So I'm drawing out this explanation as long as I possibly can in the hope that it finishes before. And it may well observe that the markets are closed right now, so it can't make trading decisions. It's possible that it will do that or it might decide that it wants to make some trading decisions anyway, even though the markets are closed. But we will soon find out

In [24]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

Error initializing MCP server: Connection closed


McpError: Connection closed

---

So here we go. It has decided that it has—we gave it the tools to do it, and so it's decided to do it. It's got a bunch of different summary actions, the results of the research and the trades that were executed. It bought shares and sold shares, and that's the current portfolio status. And it's got some next steps at the bottom. So that is the result of our trader agent running using the research agent and being able to execute its tools and also being armed with the resources that we included in the prompt. And I won't be doing it.

### Then go and look at the trace

http://platform.openai.com/traces


In [ ]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

### Now it's time to review the Python module made from this:

`mcp_params.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.

You'll notice I've done something a bit fancy with code like this:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

But it's equivalent.


In [2]:
from traders import Trader


In [3]:
trader = Trader("Ed")

In [ ]:
await trader.run()

In [ ]:
await read_accounts_resource("Ed")

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [ ]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")